In [1]:
import torch
from llama_index.llms import LlamaCpp
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import SimpleDirectoryReader
from llama_index import Document
import os
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index import VectorStoreIndex, ServiceContext, load_index_from_storage
from fastapi import FastAPI
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank

ImportError: cannot import name 'LlamaCpp' from 'llama_index.llms' (unknown location)

In [ ]:
!pip install llama-index-llms-llama-cpp

  Using cached llama_index_llms_llama_cpp-0.1.4-py3-none-any.whl.metadata (695 bytes)
  Using cached llama_cpp_python-0.2.81.tar.gz (50.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached llama_index_llms_llama_cpp-0.1.4-py3-none-any.whl (6.0 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [94 lines of output]
      *** scikit-build-core 0.9.8 using CMake 3.26.4 (wheel)
      *** Configuring CMake...
      loading initial cache file /tmp/tmpd6lpnv9f/build/CMakeInit.txt
      -- The C compiler identification is GNU 13.2.0
      -- The CXX compiler identification is GNU 13.2.0
      -- Detecting C compiler ABI info
      -- Detecting C compiler ABI info - done
      -- 

In [ ]:

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    # model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


def get_build_index(documents,llm,embed_model="local:BAAI/bge-small-en-v1.5",sentence_window_size=3,save_dir="./vector_store/index"):
  
  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

  sentence_context = ServiceContext.from_defaults(
      llm = llm,
      embed_model= embed_model,
      node_parser = node_parser,
  )

  if not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents], service_context=sentence_context
        )
        index.storage_context.persist(persist_dir=save_dir)
  else:
      # load the existing index
      index = load_index_from_storage(
          StorageContext.from_defaults(persist_dir=save_dir),
          service_context=sentence_context,
      )

  return index


def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n, model="BAAI/bge-reranker-base"
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
  )

  return engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
app = FastAPI()

@app.get("/test-llm/")
async def read_item(query: str):
    # read documents
    documents = SimpleDirectoryReader(
    input_files = ["./documents/survey_on_llms.pdf"]
    ).load_data()
    
    documents = Document(text = "\n\n".join([doc.text for doc in documents]))

    vector_index = get_build_index(documents=documents, llm=llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3, save_dir="./vector_store/index")

    query_engine = get_query_engine(sentence_index=vector_index, similarity_top_k=6, rerank_top_n=2)
    
    response = query_engine.query(query)
    
    return {"response": display_response(response)}